# 0. Setup

### Import Required Libaried


In [ ]:
import os
import openai

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

import dotenv 
#load the environment variables of .env file
%load_ext dotenv
%dotenv

### Setup Azure Credentials


In [ ]:
# Setup the required credential for using Azure cognitive search
search_endpoint = f"https://{os.getenv('AZURE_SEARCH_SERVICE')}.search.windows.net/"
search_creds = AzureKeyCredential(os.getenv("AZURE_SEARCH_KEY"))
search_client = SearchClient(endpoint=search_endpoint,
                                index_name=os.getenv("AZURE_SEARCH_INDEX"),
                                credential=search_creds)

# Setup the required credential for using Azure OpenAI
openai.api_type = "azure"
openai.api_key = os.getenv("AZURE_OPENAI_KEY")     
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = "2023-05-15"

### Setup Embedding model


In [ ]:
def compute_embedding(text):
    return openai.Embedding.create(engine="embedding", input=text)["data"][0]["embedding"]

def nonewlines(s: str) -> str:
    return s.replace(' ', ' ').replace('\r', ' ')

# 1. Using ChatGPT Through calling API

In [ ]:
#change the query to what you want to ask chatGPT
query = "Where is the capital of India?"

messages = [
    {'role' : 'user', 'content' : query }
]

chat_completion = openai.ChatCompletion.create(
    deployment_id="chat",
    model="gpt-35-turbo",
    messages=messages, 
    temperature=0.7, 
    max_tokens=1024, 
    n=1)

chat_content = chat_completion.choices[0].message.content
chat_content

### Modifying the role of ChatGPT 
- Adding system message
- Adding few shot example

In [ ]:
#change the query to what you want to ask chatGPT
query = "Write a poem about university life"

#change the systemMessage to how you want chatGPT to behave
systemMessage = '''You are a Shakespearean writing assistant who speaks in a Shakespearean style. 
                    You help people come up with creative ideas and content like stories, poems, and songs that use Shakespearean style of writing style, including words like "thou" and "hath”.
                    Here are some example of Shakespeare's style:
                    - Romeo, Romeo! Wherefore art thou Romeo?
                    - Love looks not with the eyes, but with the mind; and therefore is winged Cupid painted blind.
                    - Shall I compare thee to a summer's day? Thou art more lovely and more temperate.'''

messages = [
    {'role' : 'system', 'content' : systemMessage},
    #change the content here to your example question
    {'role' : 'user', 'content' : 'Please write a short text turning down an invitation to dinner.'},
    #change the content here to your example answer
    {'role' : 'assistant', 'content' : '''Dearest,
                                        Regretfully, I must decline thy invitation.
                                        Prior engagements call me hence. Apologies.'''},
    {'role' : 'user', 'content' : query }
]

chat_completion = openai.ChatCompletion.create(
    deployment_id="chat",
    model="gpt-35-turbo",
    messages=messages, 
    temperature=0.7, 
    max_tokens=1024, 
    n=1)

chat_content = chat_completion.choices[0].message.content
print(chat_content)

# 2. Implementing a RAG model

### Obtain related information using vector search

In [ ]:
#change the query to what you want to ask the RAG model
query = "What is SIG"
query_vector = compute_embedding(query)

r = search_client.search(query, 
                        top=3, 
                        vector=query_vector, 
                        top_k=50, 
                        vector_fields="embedding")

results = [doc["sourcepage"] + ": " + nonewlines(doc["content"]) for doc in r]

for result in results:
    print(result)

### Query the GPT model with query + information

In [ ]:
#change the systemMessage to how you want chatGPT to behave
systemMessage = """AI Assistant that helps user to answer questions from sources provided. Be brief in your answers.
                    Answer ONLY with the facts listed in the list of sources below. 
                    If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. 
                    Each source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. 
                    Use square brackets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].
                """

messages = [
    {'role' : 'system', 'content' : systemMessage},
    {'role' : 'user', 'content' : query + "   Source:" + " ".join(results)}
]

chat_completion = openai.ChatCompletion.create(
    deployment_id="chat",
    model="gpt-35-turbo",
    messages=messages, 
    temperature=0.7, 
    max_tokens=1024, 
    n=1)

chat_content = chat_completion.choices[0].message.content
print(chat_content)